In [ ]:
'''
import numpy   as np 
import scipy.misc # to visualize only  

#x = np.loadtxt("data/train_x.csv", delimiter=",") # load from text 
#y = np.loadtxt("data/train_y.csv", delimiter=",") 
#x = x.reshape(-1, 64, 64) # reshape 
#y = y.reshape(-1, 1)
np.save("data/"+"X_train",x)
np.save("data/"+"y_train",y)
print("X.shape",x.shape, "Y.shape",y.shape)
'''

In [1]:
import numpy as np
x = np.load("data/X_train.npy")
y = np.load("data/y_train.npy")

In [2]:
#preprocessing
#using 1 hot encoding encode the 40 unique classes of y 
decode = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]
encodedY = np.zeros((50000,40))
for i in range(0,49999):
    index = decode.index(y[i])
    encodedY[i][index] = 1

flatX = np.zeros((50000,4096))
for i in range(0,49999):
    flatX[i] = x[i].flatten()
    #normalize input data
    mean = np.mean(flatX[i])
    flatX[i] = (flatX[i]- mean)/128.0
    
#for early testing, use 8% as training set and 2% as testing set
inputX = flatX[0:800]
validX = flatX[800:1000]
inputY = encodedY[0:800]
validY = encodedY[800:1000]

In [64]:
from numpy import exp, array, random, dot

class FeedForwardNeuralNetwork():
    #Nlayer must be >=2
    def __init__(self,NLayer,Nneuron,ranSeed,Ninput,Noutput,bias):
        #specify the number of hidden layers , number of neurons needed per layer, number of weights per neuron
        #ranSeed specify the random seed used in initializing weights
        #activation specified what activation function is to be used, pass in 0 to use default sigmoid function
        #-------------------------------
        np.random.seed(ranSeed)
        self.bias = bias
        self.Nneuron = Nneuron
        self.Ninput = Ninput
        self.Noutput = Noutput
        self.NLayer = NLayer
        #input weights
        self.InputWeight = np.random.uniform(0,1,(Nneuron,Ninput))
        #this is weight for neurons from hidden layer 1 to (n-1)
        self.neurons = np.random.uniform(0,1,(NLayer-1,Nneuron,Nneuron))
        #store output of each layer
        self.O = np.random.uniform(0,1,(NLayer,Nneuron))
        #the weight for the last layer of neurons
        self.OutputWeight = np.random.uniform(0,1,(Nneuron,Noutput))
        #encoding output layer with multi-class domains, each output neuron represents a discrete output value
        #see as an classification task
        self.output = np.random.uniform(0,1,Noutput)
    
    #allows users to manually set all the weights
    def set_Weights(self,InputWeights,OutputWeights,HiddenWeights):
        self.OutputWeight = OutputWeights
        self.InputWeight = InputWeights
        self.neurons = HiddenWeights
    
    
    
    #default activation function, normalize input into 0 and 1
    # normalise them between 0 and 1.
    def sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    #train the neural network
    #can change learning rate
    def train(self, num_training_example, training_inputs, training_outputs, num_iteration, learning_rate):
        #use to store outputs
        outputArr = np.zeros((num_training_example,self.Noutput))
        for iteration in xrange(num_iteration):
            for i in range(0,num_training_example):
                #---------------------------------------------
                #need a output layer special gamma
                #-------F--------------------------------------
                gammaOutput = np.zeros(self.Noutput)
                # gamma is the error correction for all neurons in NN
                # gamma values are used to backpropagate
                gamma = np.zeros((self.NLayer,self.Nneuron))
                #---------------------------------------------
                # Forward pass
                self.forward(training_inputs[i])
                outputArr[i] = np.copy(self.output)
                #---------------------------------------------
                #compute corrections
                # first calculate correction in the output layer
                for k in range(0,self.Noutput):
                    gammaOutput[k] = self.sigmoid_derivative(self.output[k])*(training_outputs[i][k]-self.output[k])
                    #calculate the gamma values for the last hidden layer
                    #And update weights for output layer
                    for z in range(0,self.Nneuron):
                        gamma[self.NLayer-1][z] = self.sigmoid_derivative(self.O[self.NLayer-1][z])*dot(gammaOutput,self.OutputWeight[z])
                for z in range(0,self.Nneuron):
                    for k in range(0,self.Noutput):
                        #print ("position is " + str(z) + str(k))
                        #print (self.OutputWeight[z][k])
                        self.OutputWeight[z][k] = self.OutputWeight[z][k] + (learning_rate*gammaOutput[k]*
                                                                              self.O[self.NLayer-1][z])
                        #print (self.OutputWeight[z][k])
                        #print ((learning_rate*gammaOutput[k]*self.O[self.NLayer-1][z]))
                #--------------------------------------
                #calculate the backprop for hidden layer
                #j is counting the current layer
                #z is counting the neuron position
                #propagate gamma values
                for j in range(self.NLayer-2,-1,-1):
                    for z in range(0,self.Nneuron):
                        #multiple the derivative with all the connected neuron's gamma                
                        #SumGamma will compute all the propagated gammas
                        gamma[j][z] = self.sigmoid_derivative(self.O[j][z])*(self.SumGamma(gamma,j,z))
                        #update the weights now
                        for k in range(0,self.Nneuron):
                            self.neurons[j][z][k] = self.neurons[j][z][k] + (learning_rate*gamma[j][z]*
                            self.O[j][k])
                #---------------------------------------
        print (outputArr)
                
        
    #compute the propagated gamma of a neuron
    def SumGamma(self,gamma,Layer,position):
        output = dot(gamma[Layer+1],self.neurons[Layer][position])
        return output
                
    # pass input through all the layers until the output layer
    def forward(self, inputs):
        #first pass from input layer to hidden layer
        current = np.zeros(self.Nneuron)
        for i in range(0,self.Nneuron):
            current[i] = self.sigmoid(dot(inputs,self.InputWeight[i])+self.bias)
            self.O[0][i] = current[i]
        #passing through hidden layers
        for i in range(0,self.NLayer-1):
            for j in range(0,self.Nneuron):
                current[j] = self.sigmoid(dot(self.O[i],self.neurons[i][j])+self.bias)
            self.O[i+1] = np.copy(current)
        #now from hidden layer to output layer
        #the weight for the last hidden layer is the weight for the output layer 
        #---------------------------------------------------------------
        for i in range(0,self.Noutput):
            self.output[i]= self.sigmoid(dot(current,self.OutputWeight[:,i])+self.bias)
        #----------------------------------------------------------------
        
    #a simple function to see the validation accuracy    
    def validate(self, num_validate_example, validating_inputs, validating_outputs):
        count = 0
        for i in range(0,num_validate_example):
            self.forward(validating_inputs[i])
            if(self.output.index(1) == validating_outputs[i].index(1)):
                count = count + 1
        print("accuracy is" + str(count*1.0/num_validate_example))
    
        
        
if __name__ == "__main__":

    #Intialise a single neuron neural network.
    #def __init__(self,NLayer,Nneuron,ranSeed,Ninput,Noutput,bias):
    #neural_network = FeedForwardNeuralNetwork(3,200,1,4096,40,1)
    neural_network = FeedForwardNeuralNetwork(2,3,1,8,8,1)
    #neural_network.set_Weights(InputWeights,OutputWeights,HiddenWeights)
    trainX = [[1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0]]
    trainY = [[1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0]]
    neural_network.train(8,trainX,trainY,10000,0.01)
    #def train(self, num_training_example, training_inputs, training_outputs, num_iteration, learning_rate):
    #neural_network.train(800,inputX,inputY,20,0.001)
    #def validate(self, num_validate_example, validating_inputs, validating_outputs):
    #neural_network.validate(200,validX,validY)
    


[[ 0.12503703  0.1245883   0.12599021  0.12528974  0.12496891  0.12422922
   0.12596939  0.12434601]
 [ 0.12389137  0.12314523  0.12452901  0.12394216  0.12363384  0.12292305
   0.12461102  0.12296018]
 [ 0.12413226  0.12338768  0.12478417  0.12406249  0.12374302  0.1230027
   0.12474091  0.12342825]
 [ 0.1269825   0.12654858  0.12765206  0.12693952  0.12659277  0.1258398
   0.12760747  0.12628544]
 [ 0.12570119  0.12528005  0.12668793  0.12558016  0.12521641  0.12444327
   0.12624398  0.12496471]
 [ 0.12461189  0.12413082  0.12551964  0.1250849   0.12452318  0.12385781
   0.12546443  0.12404205]
 [ 0.12293163  0.12247739  0.123848    0.12341202  0.12312072  0.1221537
   0.12377098  0.12237054]
 [ 0.12672024  0.12625047  0.12766219  0.12702548  0.12672892  0.12600619
   0.12741241  0.12605292]]
